In [ ]:
%load_ext autoreload
%autoreload 2

In [1]:
import sys

sys.path.append("../")

In [ ]:
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "0"
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"

In [2]:
import pandas as pd
from pytorch_lightning import Trainer
from pytorch_lightning.callbacks import ModelCheckpoint
from sklearn.model_selection import train_test_split
from transformers import T5Tokenizer

In [3]:
# [pin]
file_path = "../data/data-hard.csv"
root_path = "../data/"


df = pd.read_csv(file_path)
df["prefix"] = "clsorg"
df = df.rename({"message": "input_text", "label": "target_text"}, axis=1)
df.sample(20)

,input_text,target_text,prefix
1746,#Идея 🧠 Закрытые идеи Мозгового центра Сигнало...,225-3;127-3,clsorg
1772,#Распадская. Обзор финансовых показателей по Р...,129-3,clsorg
7154,🤷‍♂«Газпром» хочет поправить свое положение за...,48-3,clsorg
6746,🛢 Лукойл - деньги есть! Отчет за 1 полугодие ...,111-4,clsorg
3106,Рынки утром 🔸Дело о банкротстве СПБ Биржи поя...,230-4,clsorg
6309,📉 Покатились вниз: до каких уровней могут упас...,152-3,clsorg
5565,🏁 Итоги дня: 13 марта 🔸 Продажи производите...,4-5;241-4,clsorg
3105,Рынки утром 🔸 СПБ биржа хочет перевести гонко...,225-4,clsorg
1775,#Теханализ 💡 Текущие ожидания отдела Теханали...,127-4,clsorg
6605,🔼 Торги на российском рынке начались в плюсе: ...,221-3;111-3;112-3;163-3;236-3,clsorg


In [4]:
m_name = "cointegrated/rut5-small"
tokenizer = T5Tokenizer.from_pretrained(m_name)

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [8]:
from src.t5.dataset import NERDataModel
from src.t5.model import NERModel

BATCH_SIZE = 64
EPOCHS = 10
train_df, test_df = train_test_split(df, test_size=0.25, random_state=42)
data_module = NERDataModel(train_df, test_df, tokenizer, batch_size=BATCH_SIZE)
data_module.setup()

In [9]:
model = NERModel()

checkpoint_callback = ModelCheckpoint(
    dirpath="checkpoints",
    filename="ner",
    save_top_k=1,
    verbose=True,
    monitor="val_loss",
    mode="min",
)

trainer = Trainer(
    callbacks=[checkpoint_callback],
    max_epochs=EPOCHS,
    accelerator="cuda",
)

Trainer will use only 1 of 8 GPUs because it is running inside an interactive / notebook environment. You may try to set `Trainer(devices=8)` but please note that multi-GPU inside interactive / notebook environments is considered experimental and unstable. Your mileage may vary.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [10]:
!rm -r lightning_logs

rm: cannot remove 'lightning_logs': No such file or directory


In [11]:
trainer.fit(model, data_module)

You are using a CUDA device ('NVIDIA A100-SXM4-80GB') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
Missing logger folder: /home/jovyan/pavlov/hacks/gagarin/hakaton-gagarin-sentiment_interface/pybooks/lightning_logs
/home/jovyan/.mlspace/envs/sbs_gen/lib/python3.11/site-packages/pytorch_lightning/callbacks/model_checkpoint.py:653: Checkpoint directory /home/jovyan/pavlov/hacks/gagarin/hakaton-gagarin-sentiment_interface/pybooks/checkpoints exists and is not empty.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3,4,5,6,7]
/home/jovyan/.mlspace/envs/sbs_gen/lib/python3.11/site-packages/transformers/optimization.py:457: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch 

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Epoch 0, global step 85: 'val_loss' reached 1.92267 (best 1.92267), saving model to '/home/jovyan/pavlov/hacks/gagarin/hakaton-gagarin-sentiment_interface/pybooks/checkpoints/ner-v2.ckpt' as top 1


...

Epoch 7, global step 680: 'val_loss' was not in top 1


Validation: |          | 0/? [00:00<?, ?it/s]

Epoch 8, global step 765: 'val_loss' was not in top 1


Validation: |          | 0/? [00:00<?, ?it/s]

Epoch 9, global step 850: 'val_loss' was not in top 1
`Trainer.fit` stopped: `max_epochs=10` reached.


In [12]:
trained_model = NERModel.load_from_checkpoint("checkpoints/ner-v2.ckpt")
trained_model.freeze()

In [13]:
import numpy as np
import pandas as pd
import torch
from torch.utils.data import Dataset, SequentialSampler, DataLoader
from tqdm.notebook import tqdm
from transformers import T5Tokenizer
import time


class CustomDataset(Dataset):
    def __init__(self, data, tokenizer, max_length):
        self.data = data
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        item = self.data.iloc[idx]
        input_text = item["prefix"] + ": " + item["input_text"]
        encoding = self.tokenizer(
            input_text,
            max_length=self.max_length,
            padding="max_length",
            truncation=True,
            return_attention_mask=True,
            add_special_tokens=True,
            return_tensors="pt",
        )
        encoding["input_ids"] = encoding["input_ids"].squeeze(0)
        return encoding


def generate_answer_batched_v2(
    trained_model: NERModel,
    tokenizer: T5Tokenizer,
    data: pd.DataFrame,
    batch_size: int = 64,
    dataloader_workers: int = 16
):
    predictions = []
    dataset = CustomDataset(data, tokenizer, max_length=396)
    sequential_sampler = SequentialSampler(dataset)
    dataloader = DataLoader(dataset, batch_size=batch_size, sampler=sequential_sampler, num_workers=dataloader_workers)
    with torch.no_grad():
        for source_encoding in tqdm(dataloader):
            t0 = time.time()
            generated_ids = trained_model.model.generate(
                input_ids=source_encoding["input_ids"].cuda(),
                attention_mask=source_encoding["attention_mask"].cuda(),
                num_beams=3,
                max_length=80,
                temperature=0.5,
                repetition_penalty=1.0,
                early_stopping=True,
                use_cache=True,
            ).cpu()
            preds = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)
            predictions.append(preds)

    return sum(predictions, [])

In [14]:
from src.t5.utils import evaluate_metric, generate_answer_batched

In [15]:
%%time

predictions = generate_answer_batched(
    trained_model=trained_model, tokenizer=tokenizer, data=test_df, batch_size=64
)

  0%|          | 0/29 [00:00<?, ?it/s]

CPU times: user 21.3 s, sys: 178 ms, total: 21.5 s
Wall time: 21.5 s


In [18]:
%%time

predictions = generate_answer_batched_v2(
    trained_model=trained_model, tokenizer=tokenizer, data=test_df, batch_size=64, dataloader_workers=16
)

  0%|          | 0/29 [00:00<?, ?it/s]

/home/jovyan/.mlspace/envs/sbs_gen/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:492: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.5` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


CPU times: user 20.2 s, sys: 656 ms, total: 20.8 s
Wall time: 21 s


In [19]:
ldf = test_df.copy()
ldf["predictions"] = predictions
ldf[["tcomp", "tsent"]] = (
    ldf["target_text"].str.split(";", expand=True)[0].str.split("-", expand=True)
)
ldf[["pcomp", "psent"]] = (
    ldf["predictions"].str.split(";", expand=True)[0].str.split("-", expand=True)
)

In [20]:
# [pin]

evaluate_metric(
    company_predictions=ldf["pcomp"].tolist(),
    company_labels=ldf["tcomp"].tolist(),
    sentiment_predictions=ldf["psent"].tolist(),
    sentiment_labels=ldf["tsent"].tolist(),
)

ValueError: invalid literal for int() with base 10: 'av'